In [1]:
# Import needed modules
import pandas as pd
import numpy as np

In [2]:
# Import the data and combine into one Dataframe
li = []
for i in range(0,56):
    li.append(pd.read_csv(f'data/Kickstarter0{str(i).zfill(2)}.csv'))
df = pd.concat(li, axis=0, ignore_index = True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 37 columns):
backers_count               209222 non-null int64
blurb                       209214 non-null object
category                    209222 non-null object
converted_pledged_amount    209222 non-null int64
country                     209222 non-null object
created_at                  209222 non-null int64
creator                     209222 non-null object
currency                    209222 non-null object
currency_symbol             209222 non-null object
currency_trailing_code      209222 non-null bool
current_currency            209222 non-null object
deadline                    209222 non-null int64
disable_communication       209222 non-null bool
friends                     300 non-null object
fx_rate                     209222 non-null float64
goal                        209222 non-null float64
id                          209222 non-null int64
is_backing                  300 

## Features that are going to be dropped:

- permissions, is_backing, is_starred, friends:   
    -> only 300 values and some of them are only NaNs     
- slug, source_url, url:  
    -> same information can be found in category and name
- creator, id, profile:  
    -> information about the creator is useless for us
- currency_symbol, currency_trailing_code, current_currency, usd_type, static_usd_rate:   
    -> redundant information
- disable_communication, is_starrable, photo, location, pledged, spotlight:  
    -> redundant information

In [4]:
# Drop features
df.drop(['permissions', 'slug', 'source_url', 'urls', 'creator', 'currency_symbol', 'currency_trailing_code', 
        'current_currency', 'usd_pledged',
        'disable_communication', 'id', 'is_starrable', 'photo', 'location', 'pledged',
        'profile', 'spotlight', 'static_usd_rate', 'usd_type', 
        'is_backing', 'is_starred', 'friends'], axis=1, inplace=True)

In [5]:
# Drop all projects that are anything but successful or failed/canceled
df = df.query("state == 'successful' or state == 'failed' or state == 'canceled'")
df.reset_index(inplace=True, drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201288 entries, 0 to 201287
Data columns (total 15 columns):
backers_count               201288 non-null int64
blurb                       201280 non-null object
category                    201288 non-null object
converted_pledged_amount    201288 non-null int64
country                     201288 non-null object
created_at                  201288 non-null int64
currency                    201288 non-null object
deadline                    201288 non-null int64
fx_rate                     201288 non-null float64
goal                        201288 non-null float64
launched_at                 201288 non-null int64
name                        201288 non-null object
staff_pick                  201288 non-null bool
state                       201288 non-null object
state_changed_at            201288 non-null int64
dtypes: bool(1), float64(2), int64(6), object(6)
memory usage: 21.7+ MB


## Reformat and extract data

- Change time format  
- Extract the category in which a project was posted  
- Create new variables for how long a project ran etc.  
- Convert goal amount to usd  
- Change class type data to 0s and 1s  
- Drop NaNs
- Create variables for blurb and name length, since plain ext is useless to us

In [7]:
# Reformat the category, so we can access the genre in which the project was posted
df.category = df.category.map(lambda x: eval(x))
df.category = df.category.map(lambda x: x['slug'].partition('/')[0])

In [8]:
#format deadline, created at, launched at, stated_changed_at

#deadline
df.deadline = pd.to_datetime(df.deadline, unit='s')
df.deadline = df.deadline.dt.date

#created at
df.created_at = pd.to_datetime(df.created_at, unit='s')
df.created_at = df.created_at.dt.date

#launched at 
df.launched_at = pd.to_datetime(df.launched_at, unit='s')
df.launched_at = df.launched_at.dt.date

#state_changed_at
df.state_changed_at = pd.to_datetime(df.state_changed_at, unit='s')
df.state_changed_at = df.state_changed_at.dt.date

In [9]:
#create new features (days_total, days_until_success, days_until_launch) 
df['days_until_launch'] = df.launched_at - df.created_at
df['days_until_success'] = df.state_changed_at - df.launched_at 
df['days_total'] = df.deadline - df.launched_at 
df['days_diff_total_success'] = df.days_total - df.days_until_success 

In [10]:
#transform dateformat to integer
df['days_total'] = df['days_total'].dt.days.astype('int16')
df['days_diff_total_success'] = df['days_diff_total_success'].dt.days.astype('int16')
df['days_until_launch'] = df['days_until_launch'].dt.days.astype('int16')
df['days_until_success'] = df['days_until_success'].dt.days.astype('int16')

#Check for negative values of days_diff_total_success
#df.query("days_diff_total_success < 0").days_diff_total_success.describe()

In [11]:
#create new column for converted goal
#goal is given in native currency, so we have to multiply it with the conversion rate for usd
df['converted_goal_amount'] = df.goal * df.fx_rate

In [12]:
# Convert state and staff_pick to 0s and 1s
df.replace(to_replace=['canceled', 'failed', 'successful'], value=[0, 0, 1], inplace=True)
df.staff_pick = df.staff_pick.astype('int16')

In [13]:
# Drop NaNs in Dataframe
df.dropna(inplace=True)

In [14]:
# Create new features blurb_length and name_length
df['blurb_length'] = df.blurb.map(lambda x: len(x))
df['name_length'] = df.name.map(lambda x: len(x))
df.drop(['blurb', 'name', 'goal', 'fx_rate'], axis=1, inplace=True)

In [15]:
# Converted goal to integer
df.converted_goal_amount = df.converted_goal_amount.astype('int64')

## Export data

- Export as one whole dataset  
- Export after split into train and test

In [16]:
# Save dataframe as a whole
df.to_csv('KickstarterData_full.csv')

In [17]:
# Split the dataframe so we have an untouched validation set for later
from sklearn.model_selection import train_test_split
X = df.drop('state', axis=1)
y = df.state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [18]:
# Combine features and target value
Trainset = pd.concat([X_train, y_train], axis=1)
Trainset = Trainset.reset_index(drop=True)
Testset = pd.concat([X_test, y_test], axis=1)
Testset = Testset.reset_index(drop=True)

In [19]:
# Export datasets
Trainset.to_csv('Kickstarter_Train.csv', index=False)
Testset.to_csv('Kickstarter_Validation.csv', index=False)

In [20]:
Trainset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150960 entries, 0 to 150959
Data columns (total 18 columns):
backers_count               150960 non-null int64
category                    150960 non-null object
converted_pledged_amount    150960 non-null int64
country                     150960 non-null object
created_at                  150960 non-null object
currency                    150960 non-null object
deadline                    150960 non-null object
launched_at                 150960 non-null object
staff_pick                  150960 non-null int16
state_changed_at            150960 non-null object
days_until_launch           150960 non-null int16
days_until_success          150960 non-null int16
days_total                  150960 non-null int16
days_diff_total_success     150960 non-null int16
converted_goal_amount       150960 non-null int64
blurb_length                150960 non-null int64
name_length                 150960 non-null int64
state                       1509

In [21]:
Trainset.head()

,backers_count,category,converted_pledged_amount,country,created_at,currency,deadline,launched_at,staff_pick,state_changed_at,days_until_launch,days_until_success,days_total,days_diff_total_success,converted_goal_amount,blurb_length,name_length,state
0,48,music,3045,US,2012-05-17,USD,2012-11-01,2012-10-03,0,2012-11-01,139,29,29,0,3000,95,59,1
1,2,film & video,1,MX,2018-01-03,MXN,2018-02-17,2018-01-08,0,2018-02-17,5,40,40,0,5180,112,7,0
2,100,film & video,21166,US,2018-07-20,USD,2018-10-20,2018-09-20,1,2018-10-20,62,30,30,0,20000,132,58,1
3,102,art,4613,US,2018-04-29,USD,2018-06-01,2018-05-04,0,2018-06-01,5,28,28,0,455,102,41,1
4,13,fashion,374,US,2016-06-20,USD,2016-07-22,2016-06-22,0,2016-07-22,2,30,30,0,12500,134,56,0
